In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(suppress=True, linewidth=280)

from tqdm import trange
import torch
from muzero import Muzero
from environments import TicTacToe
from mcts import Tree
from network import Network

import random


In [2]:
M = Muzero(TicTacToe) #creating the model

nb_opti_steps = 1000
nb_cycles = 10 #I'm open to changing names, wasn't sure how to call it otherwise
nb_leaves_per_move = 50
nb_episodes = 20

In [4]:
#traininig model
for _ in range(nb_cycles):
    M.REPLAY_BUFFER = []
    
    for _ in trange(nb_episodes): #playing
        with torch.no_grad():
            M.REPLAY_BUFFER.append(M.mcts.full_episode(nb_leaves_per_move))
            
    for _ in trange(nb_opti_steps): #optimisation 
        M.optimize_step()
    #M.network.save('./models/ttt_candidate1.ptc') 

  0%|          | 0/1000 [00:00<?, ?it/s]/home/alex/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
100%|██████████| 1000/1000 [00:08<00:00, 112.46it/s]


In [8]:
M.network.save('./models/m_201125_a.ptc')
#M.network.load('./models/mymodel')

In [11]:
np.random(seed(42))

nb_games = 1

net = Network(TicTacToe.num_observations)
net.load('models/tictactoe_best')
tree = Tree(net, TicTacToe)


    for _ in range(nb_games):
        game = TicTacToe()
        tree.__init__(net, TicTacToe)
        
        if random.randint(0,1):
            ai_move = tree.search()
            game.play(ai_move)
            game.show

        while True:
            human_move = input("insert move [1-9]:")
            game.play(human_move-1)
            game.show
            if game.end:
                break

            ai_move = tree.search()
            game.play(ai_move)
            game.show
            if game.end:
                break
        

IndentationError: unexpected indent (<ipython-input-11-45fb74eace7a>, line 10)